# CSC421 Fall 2025 Assignment 3
### Instructor: Brandon Haworth
#### Notebook Credit: George Tzanetakis
Jupyter Notebooks you encounter during the course were largely developed by Prof. Tzanetakis from a previous iteration of this course. I've since changed/developed them where necessary for my own iterations of CSC 421.


This notebook is based on the topics covered in **Chapter 12 - Quantifying Uncertainty** and **Chapter 13 Probabilistic Reasoning** from the book *Artificial Intelligence: A Modern Approach.*  

The assignment structure is as follows:

1. [6 Marks]: Non-transitive dice war (Basic)   
2. [6 Marks]: Text Categorization setup (Basic)
3. [8 Marks]: Text Classification (Expected)
4. [6 Marks]: Specifying a Bayesian Network (Basic)
5. [6 Marks]: Inference on a Bayesian Network (Expected)

# Question 1 (Basic)  Non-transitive dice war - 6 Marks

In this question, we will explore generating samples of discrete random variables and the fascinating concept of non-transitive dice. A dice war is a game in which two dice with
different probability distributions are randomly sampled, the corresponding
samples are compared, and the one with the highest number is counted as a "win". If a dice $A$ wins on average more than half of the time against another dice $B$ we say that dice $A$ beats dice $B$.

For example let's consider a standard dice $A$ with probability distribution: $P(A) = <\frac{1}{6}, \frac{1}{6}, \frac{1}{6}, \frac{1}{6}, \frac{1}{6}, \frac{1}{6}>$ and a dice $B$ that has 3 faces with the number $4$ and three faces with the number $5$ so that $P(B) = <0, 0, 0, \frac{3}{6}, \frac{3}{6}, 0>$.  

In the cell below, code is provided for defining random variables by providing an array of values and an array of corresponding probabilities. Complete the function *dice_war* based on the code and documentation provided in the cell below. Show that the dice $B$ described above wins on average the war against dice $A$.

Now consider the following three dice/random variables $Red, Green, Blue$. They all have six faces
but different values. The corresponding values are:

1. $[2,2,4,4,9,9]$ for $Red$
2. $[1,1,6,6,8,8]$ for $Green$
3. $[3,3,5,5,7,7]$ for $Blue$

<img src="IntransitiveDice.png" width="25%">

Using the dice war function you wrote show the counter-intuitive result that the $Red$ die beats the $Green$ die, the $Green$ die beats the $Blue$ die, but the $Blue$ die beats the $Red$ die.




In [28]:
import numpy as np
from scipy import stats

class Random_Variable:

    def __init__(self, name, values, probability_distribution):
        self.name = name
        self.values = values
        self.probability_distribution = probability_distribution
        if all(type(item) is np.int64 for item in values):
            self.type = 'numeric'
            self.rv = stats.rv_discrete(name = name, values = (values, probability_distribution))
        elif all(type(item) is str for item in values):
            self.type = 'symbolic'
            self.rv = stats.rv_discrete(name = name, values = (np.arange(len(values)), probability_distribution))
            self.symbolic_values = values
        else:
            self.type = 'undefined'

    def sample(self,size):
        if (self.type =='numeric'):
            return self.rv.rvs(size=size)
        elif (self.type == 'symbolic'):
            numeric_samples = self.rv.rvs(size=size)
            mapped_samples = [self.values[x] for x in numeric_samples]
            return mapped_samples

    def get_name(self):
        return self.name


def dice_war(A,B, num_samples = 1000, output=True):
    # YOUR CODE GOES HERE
    samples_A = A.sample(num_samples)
    samples_B = B.sample(num_samples)
    prob = np.mean(samples_A > samples_B)
    res = prob > 0.5

    if output:
        if res:
            print('{} beats {} with probability {}'.format(A.get_name(),
                                                           B.get_name(),
                                                           prob))
        else:
            print('{} beats {} with probability {:.2f}'.format(B.get_name(),
                                                               A.get_name(),
                                                               1.0-prob))
    return (res, prob)


# Example: Create two dice from the example above A and B
values = np.arange(1,7,dtype=np.int64)
probabilities_A = np.array([1/6., 1/6., 1/6., 1/6., 1/6., 1/6.])
probabilities_B = np.array([0/6., 0/6., 0/6., 3/6., 3/6., 0/6.])

dieA = Random_Variable('DieA', values, probabilities_A)
dieB = Random_Variable('DieB', values, probabilities_B)

(res, prob)=dice_war(dieA,dieB)

# --- Non-transitive dice setup ---
values = np.arange(6, dtype=np.int64)
prob = np.array([1/6.]*6)

# just keep the mapping in mind when interpreting results
red = Random_Variable('Red', values, prob)
green = Random_Variable('Green', values, prob)
blue = Random_Variable('Blue', values, prob)

# store face value arrays separately
red_faces = np.array([2,2,4,4,9,9])
green_faces = np.array([1,1,6,6,8,8])
blue_faces = np.array([3,3,5,5,7,7])

# Call dice_war_mapped with proper labels for clean output
def dice_war_named(nameA, A_faces, nameB, B_faces, num_samples=10000):
    A_samples = np.random.choice(A_faces, size=num_samples)
    B_samples = np.random.choice(B_faces, size=num_samples)
    prob = np.mean(A_samples > B_samples)
    if prob > 0.5:
        print(f"{nameA} beats {nameB} with probability {prob:.3f}")
    else:
        print(f"{nameB} beats {nameA} with probability {1-prob:.3f}")

dice_war_named("Red", [2,2,4,4,9,9], "Green", [1,1,6,6,8,8])
dice_war_named("Green", [1,1,6,6,8,8], "Blue", [3,3,5,5,7,7])
dice_war_named("Blue", [3,3,5,5,7,7], "Red", [2,2,4,4,9,9])


# YOUR CODE GOES HERE
# Add code here to show the non-transitive nature of Red, Green, and Blue dice
# Create three dice Red Green Blue
# Add a tiny noise to make faces unique (works same statistically)

# Your output from this cell should look something like this, NOTE that the numbers will differ because of sampling, but the outcome should be the same
# DieB beats DieA with probability 0.75
# Red beats Green with probability 0.545
# Green beats Blue with probability 0.551
# Blue beats Red with probability 0.58

DieB beats DieA with probability 0.76
Red beats Green with probability 0.558
Green beats Blue with probability 0.556
Blue beats Red with probability 0.549


# Question 2 (Basic) - Text categorization setup - 6 Marks

Text categorization is the task of assigning a given document to one of a fixed set of categories on the basis of the text it contains. Naive Bayes models are often used for this task. In these models, the query variable is the document category, and the effect variables are the presence/absence of each word in the language; the assumption is that words occur independently in documents within a given category (conditional independence), with frequencies determined by the document category. Download the following file: http://www.cs.cornell.edu/People/pabo/movie-review-data/review_polarity.tar.gz containing a dataset that has been used for text mining consisting of movie reviews classified into negative and positive. You will see that there are two folders for the positive and negative categories, and they each contain multiple text files with the reviews. You can find more information about the dataset at: http://www.cs.cornell.edu/People/pabo/movie-review-data/

Our goal will be to build a simple Bernoulli Naive Bayes classifier for this dataset. More complicated approaches using term frequency and inverse document frequency weighting and many more words are possible, but the basic concepts are the same. The goal is to understand the whole process, so DO NOT use existing machine learning packages but rather build the classifier from scratch.

Our feature vector representation for each text file will be simply a binary vector (hence Bernoulli) that shows which of the following words are present in the text file: awful bad boring dull effective enjoyable great hilarious adoxography. For example, the text file cv996_11592.txt would be represented as (0, 0, 0, 0, 1, 0, 1, 0, 0) because it contains Effective and Great but none of the other words. I've added a word here that does not exist in the dataset. Because of this, you need to ensure you have correctly implemented Laplace smoothing. For Benoulli Bayes this is, $\frac{N_{wC} + \alpha}{N_{C} + |V|\alpha}$, where $N_{wC}$ is the number of examples with this word in this class, $N_{C}$ is the total number of examples in this class, $V$ is the vocabulary so $|V|$ is the size of the vocabulary, and $\alpha$ is the smoothing value, for Laplace Smoothing this is $\alpha = 1$.

Your job is to write code that parses the text files and calculates probabilities for each dictionary word given the review polarity:

1. Write a function that can generate the feature vector of a single file given a path to that file. As a test you should be able to generate the example above for the file review_polarity/txt_sentoken/pos/cv996_11592.txt
   ```python
   def get_feature_vector(path):
   ```
   *HINT: for reading files in all at once using python:*
    ```python
   f = open(path, "r")
   contents = f.read()
   ```
2. Write a function that can generate the conditional probabilities of each word given in each class, given the directory of the data for a class. You must implement Laplace Smoothing for Bernoulli Naive Bayes correctly.
   ```python
   def word_probabilities(directory):
   ```

NOTE: os.scandir(directory), filename.is_file(), open(filename.path, "r") are useful here!

In [10]:
from google.colab import files
import zipfile, io

# Upload zip file
uploaded = files.upload()

# Extract it (assuming your file is review_polarity.zip)
with zipfile.ZipFile(io.BytesIO(uploaded['review_polarity.zip']), 'r') as zip_ref:
    zip_ref.extractall()
print("✅ Folder extracted successfully!")



Saving review_polarity.zip to review_polarity.zip
✅ Folder extracted successfully!


In [11]:
import os
import numpy as np

# YOUR CODE GOES HERE
import os
import numpy as np

# Define the fixed vocabulary (the words we’ll track)
VOCAB = ["awful", "bad", "boring", "dull", "effective",
         "enjoyable", "great", "hilarious", "adoxography"]

# --- 1. Generate the binary feature vector for a single file ---
def get_feature_vector(path):
    """
    Returns a binary numpy array of length |VOCAB|,
    where each element indicates whether the corresponding
    word is present in the text file.
    """
    with open(path, "r", encoding="utf-8") as f:
        contents = f.read().lower()  # make lowercase for consistency
    features = np.zeros(len(VOCAB), dtype=int)

    for i, word in enumerate(VOCAB):
        if word in contents:
            features[i] = 1
    return features


# --- 2. Compute conditional probabilities for each word given the class ---
def word_probabilities(directory, alpha=1):
    """
    Calculates P(word | class) for each word in VOCAB,
    using Laplace smoothing for Bernoulli Naive Bayes.
    """
    files = [f for f in os.scandir(directory) if f.is_file()]
    N_C = len(files)  # number of examples in this class
    N_wC = np.zeros(len(VOCAB))  # count of documents in class containing each word

    # Count how many documents contain each word
    for file in files:
        vec = get_feature_vector(file.path)
        N_wC += vec  # add 1 if the word appears in the file

    # Apply Laplace smoothing:
    # P(word | class) = (N_wC + α) / (N_C + 2α)
    # For Bernoulli NB, each word can appear (1) or not (0)
    probs = (N_wC + alpha) / (N_C + 2 * alpha)

    return probs


# Test cases
example_vec = get_feature_vector('review_polarity/txt_sentoken/pos/cv996_11592.txt')
neg_probs = word_probabilities('review_polarity/txt_sentoken/neg')
pos_probs = word_probabilities('review_polarity/txt_sentoken/pos')

print("Example pos/cv996_11592.txt: ", example_vec)
print("Negative vocabulary probabilities: ", neg_probs)
print(pos_probs)

# Expected output - note that numbers may vary depending on how you parse but it should not be by much
# [0 0 0 0 1 0 1 0 0]
# [0.12190287 0.54112983 0.17443013 0.10109019 0.08622398 0.05450942
#  0.31813677 0.05946482 0.00099108]
# [0.03468781 0.27849356 0.05450942 0.02576809 0.15361744 0.09613479
#  0.48166501 0.13181368 0.00099108]

Example pos/cv996_11592.txt:  [0 0 0 0 1 0 1 0 0]
Negative vocabulary probabilities:  [0.12275449 0.54491018 0.1756487  0.10179641 0.08682635 0.05489022
 0.32035928 0.05988024 0.000998  ]
[0.03493014 0.28043912 0.05489022 0.0259481  0.15469062 0.09680639
 0.48502994 0.13273453 0.000998  ]


# QUESTION 3 (EXPECTED) - Text classification  - 8 Marks

Write Python code for classifying a particular test instance (in our case, a movie review) following a Bernoulli Naive Bayes approach, i.e. you just model the presence/absence of each word. NOTE: because we are not counting word occurences we are simply using a binary presence feature vector (the word appears or does not apepar in a document), we can use the Bernoulli approach. Your code should calculate the likelihood that the review is positive, given the corresponding conditional probabilities for each dictionary word, as well as the likelihood that the review is negative, given the corresponding conditional probabilities for each dictionary word. Check that your code works by providing a few example cases of prediction. Your code should be written from "scratch" and only use NumPy but **NOT** machine learning libraries like scikit-learn, tensorflow or pytorch.

You should write three functions:

1. A function that computes the likelihood given the path to a file and the conditional probabilities for a particular class. In Bernoulli Bayes we care about the exclusion terms, i.e. the likelihood is $ p(\mathbf{x} | C_k) = \prod_{i = 0}^{n}  p_{k_i}^{x_i}(1 - p_{k_i})^{1 - x_i}$ where $\mathbf{x}$ is the feature vector for a particular file, $C_k$ is the class k (for us there are two pos and neg), $n$ is the number of vocabulary items, $p_{k_i}$ is the porbability of the vocabulary word $i$ appearing in class $k$, $x_i$ is the $i'th$ entry in the feature vector $\mathbf{x}$. Note how $x_i$ is essentialy an indicator function so that the likelihood accounts for the entire vocabulary even if the word is not present. In that case we get the exclusion probability $1 - p_{k_i}$. Bernoulli Bayes is interesting in that way!
   ```python
   def likelihood(path, probs):
   ```
2. A function that computes the class priors given two directories, and returns the results as a list [$p(C_1)$, $p(C_2)$]. Yes we are aware this is a nicely balanced datset where the priors are 50/50, however you can not assume that!
   ```python
   def class_priors(class_dir_1, class_dir_2):
   ```
3. A function that returns the class prediction as a string 'positive' or 'negative' depending on the bernouli bayes outcome, NOTE: since we only care about argmax, and the denominator is the same for both classes, you can ignore the denominator. That is your predict function should be $\underset{k \in \{1, \ldots, K\}}{\operatorname{argmax}} p(C_k)p(\mathbf{x} | C_k)$, note for us $k$ is $2$ because we just have a positive and a negative class.
   ```python
   def predict(path, neg_probs, pos_probs, neg_prior, pos_prior):
   ```
4. A function that returns the accuracy of the classifier. Note we would normally, at the very least, split the data into training and test sets. However, here I'd like you write a function that takes two directories, classifies all the files and computes the overall accuracy. Accuracy is $\frac{TP + TN}{TP + TN + FP + FN}$ where $TP, TN, FP, FN$ are true positives, true ngatives, false positives, and false negatives respectively, i.e., correct classiffications over all classifications.
   ```python
   def accuracy(neg_dir, pos_dir):
   ```


In [13]:
# YOUR CODE GOES HERE

# Compute probabilities for both classes
neg_probs = word_probabilities('review_polarity/txt_sentoken/neg')
pos_probs = word_probabilities('review_polarity/txt_sentoken/pos')

# Compute class priors
neg_prior, pos_prior = class_priors('review_polarity/txt_sentoken/neg',
                                    'review_polarity/txt_sentoken/pos')

# --- Likelihood function ---
def likelihood(path, probs):
    """
    Computes the likelihood P(x|Ck) for a file given the class word probabilities.
    Uses Bernoulli Naive Bayes formula:
        P(x|Ck) = ∏ p_i^x_i * (1 - p_i)^(1 - x_i)
    """
    x = get_feature_vector(path)  # binary vector
    p = probs                     # P(word|class)

    # To avoid underflow, use log probabilities
    log_likelihood = np.sum(x * np.log(p) + (1 - x) * np.log(1 - p))
    return np.exp(log_likelihood)  # return actual likelihood


# --- Class priors function ---
def class_priors(class_dir_1, class_dir_2):
    """
    Computes priors P(C1) and P(C2) from the number of documents in each class.
    """
    N1 = len([f for f in os.scandir(class_dir_1) if f.is_file()])
    N2 = len([f for f in os.scandir(class_dir_2) if f.is_file()])
    total = N1 + N2
    return [N1 / total, N2 / total]


# --- Pediction function ---
def predict(path, neg_probs, pos_probs, neg_prior, pos_prior):
    """
    Classifies a review as 'positive' or 'negative' using Bernoulli Naive Bayes.
    We compare:  P(neg)*P(x|neg)  vs  P(pos)*P(x|pos)
    """
    # use log-space to avoid numeric underflow
    x = get_feature_vector(path)

    log_neg = np.sum(x * np.log(neg_probs) + (1 - x) * np.log(1 - neg_probs)) + np.log(neg_prior)
    log_pos = np.sum(x * np.log(pos_probs) + (1 - x) * np.log(1 - pos_probs)) + np.log(pos_prior)

    if log_pos > log_neg:
        return "positive"
    else:
        return "negative"


# ---Accuracy function ---
def accuracy(neg_dir, pos_dir):
    """
    Classifies all reviews in both directories and computes accuracy.
    Accuracy = (TP + TN) / (TP + TN + FP + FN)
    """
    neg_probs = word_probabilities(neg_dir)
    pos_probs = word_probabilities(pos_dir)
    neg_prior, pos_prior = class_priors(neg_dir, pos_dir)

    correct = 0
    total = 0

    # Check negative reviews
    for f in os.scandir(neg_dir):
        if f.is_file():
            pred = predict(f.path, neg_probs, pos_probs, neg_prior, pos_prior)
            if pred == "negative":
                correct += 1
            total += 1

    # Check positive reviews
    for f in os.scandir(pos_dir):
        if f.is_file():
            pred = predict(f.path, neg_probs, pos_probs, neg_prior, pos_prior)
            if pred == "positive":
                correct += 1
            total += 1

    return correct / total

# Some testcases with correct and incorrect classifications
path = 'review_polarity/txt_sentoken/pos/cv996_11592.txt'
print(path, 'classified as', predict(path, neg_probs, pos_probs, neg_prior, pos_prior))

path = 'review_polarity/txt_sentoken/pos/cv000_29590.txt'
print(path, 'classified as', predict(path, neg_probs, pos_probs, neg_prior, pos_prior))

path = 'review_polarity/txt_sentoken/neg/cv001_19502.txt'
print(path, 'classified as', predict(path, neg_probs, pos_probs, neg_prior, pos_prior))

path = 'review_polarity/txt_sentoken/neg/cv000_29416.txt'
print(path, 'classified as', predict(path, neg_probs, pos_probs, neg_prior, pos_prior))

print("Overall Accuracy: ", accuracy('review_polarity/txt_sentoken/neg', 'review_polarity/txt_sentoken/pos') * 100., "%")

# Expected Output
# review_polarity/txt_sentoken/pos/cv996_11592.txt classified as positive
# review_polarity/txt_sentoken/pos/cv000_29590.txt classified as negative
# review_polarity/txt_sentoken/neg/cv001_19502.txt classified as positive
# review_polarity/txt_sentoken/neg/cv000_29416.txt classified as negative
# Overall Accuracy:  67.05 %

review_polarity/txt_sentoken/pos/cv996_11592.txt classified as positive
review_polarity/txt_sentoken/pos/cv000_29590.txt classified as negative
review_polarity/txt_sentoken/neg/cv001_19502.txt classified as positive
review_polarity/txt_sentoken/neg/cv000_29416.txt classified as negative
Overall Accuracy:  67.05 %


# Question 4 (Basic)  - Specifying a Bayesian Network - 6 Marks

<img src="dyspnea.png">

Using the conventions for DBNs used in probability.ipynb (from the AIMA authors) encode the dyspnea network shown above, note we've provided the code for this below. Once you have constructed the Bayesian network display the cpt for the Lung Cancer Node (using the API provided not just showing the numbers).

The cell below contains the code that defines BayesNodes and BayesNetworks and the following cell
shows an example of defining the Burglary network and performing a query using direct enumeration and rejection sampling.

In [14]:
import numpy as np
import random

def extend(s, var, val):
    """Copy dict s and extend it by setting var to val; return copy."""
    return {**s, var: val}

def event_values(event, variables):
    """Return a tuple of the values of variables in event.
    >>> event_values ({'A': 10, 'B': 9, 'C': 8}, ['C', 'A'])
    (8, 10)
    >>> event_values ((1, 2), ['C', 'A'])
    (1, 2)
    """
    if isinstance(event, tuple) and len(event) == len(variables):
        return event
    else:
        return tuple([event[var] for var in variables])

def probability(p):
    """Return true with probability p."""
    return p > random.uniform(0.0, 1.0)

class ProbDist:
    """A discrete probability distribution. You name the random variable
    in the constructor, then assign and query probability of values.
    >>> P = ProbDist('Flip'); P['H'], P['T'] = 0.25, 0.75; P['H']
    0.25
    >>> P = ProbDist('X', {'lo': 125, 'med': 375, 'hi': 500})
    >>> P['lo'], P['med'], P['hi']
    (0.125, 0.375, 0.5)
    """

    def __init__(self, var_name='?', freq=None):
        """If freq is given, it is a dictionary of values - frequency pairs,
        then ProbDist is normalized."""
        self.prob = {}
        self.var_name = var_name
        self.values = []
        if freq:
            for (v, p) in freq.items():
                self[v] = p
            self.normalize()

    def __getitem__(self, val):
        """Given a value, return P(value)."""
        try:
            return self.prob[val]
        except KeyError:
            return 0

    def __setitem__(self, val, p):
        """Set P(val) = p."""
        if val not in self.values:
            self.values.append(val)
        self.prob[val] = p

    def normalize(self):
        """Make sure the probabilities of all values sum to 1.
        Returns the normalized distribution.
        Raises a ZeroDivisionError if the sum of the values is 0."""
        total = sum(self.prob.values())
        if not np.isclose(total, 1.0):
            for val in self.prob:
                self.prob[val] /= total
        return self

    def show_approx(self, numfmt='{:.3g}'):
        """Show the probabilities rounded and sorted by key, for the
        sake of portable doctests."""
        return ', '.join([('{}: ' + numfmt).format(v, p) for (v, p) in sorted(self.prob.items())])

    def __repr__(self):
        return "P({})".format(self.var_name)


class BayesNode:
    """A conditional probability distribution for a boolean variable,
    P(X | parents). Part of a BayesNet."""

    def __init__(self, X, parents, cpt):
        """X is a variable name, and parents a sequence of variable
        names or a space-separated string. cpt, the conditional
        probability table, takes one of these forms:

        * A number, the unconditional probability P(X=true). You can
          use this form when there are no parents.

        * A dict {v: p, ...}, the conditional probability distribution
          P(X=true | parent=v) = p. When there's just one parent.

        * A dict {(v1, v2, ...): p, ...}, the distribution P(X=true |
          parent1=v1, parent2=v2, ...) = p. Each key must have as many
          values as there are parents. You can use this form always;
          the first two are just conveniences.

        In all cases the probability of X being false is left implicit,
        since it follows from P(X=true).

        >>> X = BayesNode('X', '', 0.2)
        >>> Y = BayesNode('Y', 'P', {T: 0.2, F: 0.7})
        >>> Z = BayesNode('Z', 'P Q',
        ...    {(T, T): 0.2, (T, F): 0.3, (F, T): 0.5, (F, F): 0.7})
        """
        if isinstance(parents, str):
            parents = parents.split()

        # We store the table always in the third form above.
        if isinstance(cpt, (float, int)):  # no parents, 0-tuple
            cpt = {(): cpt}
        elif isinstance(cpt, dict):
            # one parent, 1-tuple
            if cpt and isinstance(list(cpt.keys())[0], bool):
                cpt = {(v,): p for v, p in cpt.items()}

        assert isinstance(cpt, dict)
        for vs, p in cpt.items():
            assert isinstance(vs, tuple) and len(vs) == len(parents)
            assert all(isinstance(v, bool) for v in vs)
            assert 0 <= p <= 1

        self.variable = X
        self.parents = parents
        self.cpt = cpt
        self.children = []

    def p(self, value, event):
        """Return the conditional probability
        P(X=value | parents=parent_values), where parent_values
        are the values of parents in event. (event must assign each
        parent a value.)
        >>> bn = BayesNode('X', 'Burglary', {T: 0.2, F: 0.625})
        >>> bn.p(False, {'Burglary': False, 'Earthquake': True})
        0.375"""
        assert isinstance(value, bool)
        ptrue = self.cpt[event_values(event, self.parents)]
        return ptrue if value else 1 - ptrue

    def sample(self, event):
        """Sample from the distribution for this variable conditioned
        on event's values for parent_variables. That is, return True/False
        at random according with the conditional probability given the
        parents."""
        return probability(self.p(True, event))

    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))


class BayesNet:
    """Bayesian network containing only boolean-variable nodes."""

    def __init__(self, node_specs=None):
        """Nodes must be ordered with parents before children."""
        self.nodes = []
        self.variables = []
        node_specs = node_specs or []
        for node_spec in node_specs:
            self.add(node_spec)

    def add(self, node_spec):
        """Add a node to the net. Its parents must already be in the
        net, and its variable must not."""
        node = BayesNode(*node_spec)
        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)
        self.nodes.append(node)
        self.variables.append(node.variable)
        for parent in node.parents:
            self.variable_node(parent).children.append(node)

    def variable_node(self, var):
        """Return the node for the variable named var.
        >>> burglary.variable_node('Burglary').variable
        'Burglary'"""
        for n in self.nodes:
            if n.variable == var:
                return n
        raise Exception("No such variable: {}".format(var))

    def variable_values(self, var):
        """Return the domain of var."""
        return [True, False]

    def __repr__(self):
        return 'BayesNet({0!r})'.format(self.nodes)


def enumerate_all(variables, e, bn):
    """Return the sum of those entries in P(variables | e{others})
    consistent with e, where P is the joint distribution represented
    by bn, and e{others} means e restricted to bn's other variables
    (the ones other than variables). Parents must precede children in variables."""
    if not variables:
        return 1.0
    Y, rest = variables[0], variables[1:]
    Ynode = bn.variable_node(Y)
    if Y in e:
        return Ynode.p(e[Y], e) * enumerate_all(rest, e, bn)
    else:
        return sum(Ynode.p(y, e) * enumerate_all(rest, extend(e, Y, y), bn)
                   for y in bn.variable_values(Y))

def enumeration_ask(X, e, bn):
    """
    [Figure 14.9]
    Return the conditional probability distribution of variable X
    given evidence e, from BayesNet bn.
    >>> enumeration_ask('Burglary', dict(JohnCalls=T, MaryCalls=T), burglary
    ...  ).show_approx()
    'False: 0.716, True: 0.284'"""
    assert X not in e, "Query variable must be distinct from evidence"
    Q = ProbDist(X)
    for xi in bn.variable_values(X):
        Q[xi] = enumerate_all(bn.variables, extend(e, X, xi), bn)
    return Q.normalize()

def consistent_with(event, evidence):
    """Is event consistent with the given evidence?"""
    return all(evidence.get(k, v) == v for k, v in event.items())

def prior_sample(bn):
    """
    [Figure 14.13]
    Randomly sample from bn's full joint distribution.
    The result is a {variable: value} dict.
    """
    event = {}
    for node in bn.nodes:
        event[node.variable] = node.sample(event)
    return event

def rejection_sampling(X, e, bn, N=10000):
    """
    [Figure 14.14]
    Estimate the probability distribution of variable X given
    evidence e in BayesNet bn, using N samples.
    Raises a ZeroDivisionError if all the N samples are rejected,
    i.e., inconsistent with e.
    >>> random.seed(47)
    >>> rejection_sampling('Burglary', dict(JohnCalls=T, MaryCalls=T),
    ...   burglary, 10000).show_approx()
    'False: 0.7, True: 0.3'
    """
    counts = {x: 0 for x in bn.variable_values(X)}  # bold N in [Figure 14.14]
    for j in range(N):
        sample = prior_sample(bn)  # boldface x in [Figure 14.14]
        if consistent_with(sample, e):
            counts[sample[X]] += 1
    return ProbDist(X, counts)

def weighted_sample(bn, e):
    """
    Sample an event from bn that's consistent with the evidence e;
    return the event and its weight, the likelihood that the event
    accords to the evidence.
    """
    w = 1
    event = dict(e)  # boldface x in [Figure 14.15]
    for node in bn.nodes:
        Xi = node.variable
        if Xi in e:
            w *= node.p(e[Xi], event)
        else:
            event[Xi] = node.sample(event)
    return event, w

def likelihood_weighting(X, e, bn, N=10000):
    """
    [Figure 14.15]
    Estimate the probability distribution of variable X given
    evidence e in BayesNet bn.
    >>> random.seed(1017)
    >>> likelihood_weighting('Burglary', dict(JohnCalls=T, MaryCalls=T),
    ...   burglary, 10000).show_approx()
    'False: 0.702, True: 0.298'
    """
    W = {x: 0 for x in bn.variable_values(X)}
    for j in range(N):
        sample, weight = weighted_sample(bn, e)  # boldface x, w in [Figure 14.15]
        W[sample[X]] += weight
    return ProbDist(X, W)


In [15]:
# Example of some sampling of a Bayes Node

from collections import Counter
bn = BayesNode('X', 'Burglary', {True: 0.2, False: 0.625})

bn.p(True, {'Burglary': False, 'Earthquake': True})

samples = []
for i in range(0,10000):
    samples.append(bn.sample({'Burglary': True, 'Earthquake': True}))
print(Counter(samples))


Counter({False: 7957, True: 2043})


In [16]:
# Example of a BayesNet and some queries

burglary = BayesNet([
        ('Burglary', '', 0.001),
        ('Earthquake', '', 0.002),
        ('Alarm', ['Burglary', 'Earthquake'],
         {(True, True): 0.95, (True, False): 0.94, (False, True): 0.29, (False, False): 0.001}),
        ('JohnCalls', 'Alarm', {True: 0.90, False: 0.05}),
        ('MaryCalls', 'Alarm', {True: 0.70, False: 0.01})
    ])
print(burglary.variable_node('Alarm').cpt)
ans_dist = enumeration_ask('Burglary', {'JohnCalls': True, 'MaryCalls': True}, burglary)
print(ans_dist[True],ans_dist[False])
print(rejection_sampling('Burglary', dict(JohnCalls=True, MaryCalls=True), burglary, 10000).show_approx())


{(True, True): 0.95, (True, False): 0.94, (False, True): 0.29, (False, False): 0.001}
0.2841718353643929 0.7158281646356071
False: 0.833, True: 0.167


In [17]:
# YOUR CODE GOES HERE
# Name your Bayes network for Dispnea  and specify it below

dyspnea = BayesNet([
    ('VisitToAsia', '', 0.01),  # P(Asia=True)
    ('Smoking', '', 0.5),       # P(Smoking=True)

    ('Tuberculosis', 'VisitToAsia', {True: 0.05, False: 0.01}),
    ('LungCancer', 'Smoking', {True: 0.10, False: 0.01}),

    ('Either', ['Tuberculosis', 'LungCancer'],
        {(True, True): 1.0, (True, False): 1.0, (False, True): 1.0, (False, False): 0.0}),

    ('XRay', 'Either', {True: 0.98, False: 0.05}),
    ('Dyspnea', 'Either', {True: 0.90, False: 0.30})
])

# Display the CPT for the LungCancer node
print(dyspnea.variable_node('LungCancer').cpt)


# Expected output
# {(True,): 0.1, (False,): 0.01}

{(True,): 0.1, (False,): 0.01}


# Question 5 (Expected) - Querying the Bayesian Network - 6 Marks

Answer the following queries using exact inference with enumeration and approximate inference of the same queries using both rejection sampling and likelihood weighting:

1. given that a patient has tuberculosis, what is the likelihood of being a smoker?
2. given that a patient has been in Asia and has a positive x-ray, what is the likelihood of having dyspnea?
3. given that a patient is a smoker and has lung cancer, what is the likelihood of having dyspnea?

Use 100000 samples where needed

In [18]:
# YOUR CODE GOES HERE


# Query 1: Likelihood of Smoker given Tuberculosis
exact_1 = enumeration_ask('Smoking', {'Tuberculosis': True}, dyspnea)
reject_1 = rejection_sampling('Smoking', {'Tuberculosis': True}, dyspnea, 100000)
weight_1 = likelihood_weighting('Smoking', {'Tuberculosis': True}, dyspnea, 100000)

print("Likelihood of Smoker given Tuberculosis using exact inference: ", exact_1[True])
print("Likelihood of Smoker given Tuberculosis using approximate rejection sampling: ", reject_1[True])
print("Likelihood of Smoker given Tuberculosis using approximate likelihood weighting: ", weight_1[True])


# Query 2: Likelihood of Dyspnea given VisitToAsia and Positive XRay
exact_2 = enumeration_ask('Dyspnea', {'VisitToAsia': True, 'XRay': True}, dyspnea)
reject_2 = rejection_sampling('Dyspnea', {'VisitToAsia': True, 'XRay': True}, dyspnea, 100000)
weight_2 = likelihood_weighting('Dyspnea', {'VisitToAsia': True, 'XRay': True}, dyspnea, 100000)

print("Likelihood of Dyspnea given VisitToAsia and Positive XRay using exact inference: ", exact_2[True])
print("Likelihood of Dyspnea given VisitToAsia and Positive XRay using approximate rejection sampling: ", reject_2[True])
print("Likelihood of Dyspnea given VisitToAsia and Positive XRay using approximate likelihood weighting: ", weight_2[True])


# Query 3: Likelihood of Dyspnea given LungCancer and Smoker
exact_3 = enumeration_ask('Dyspnea', {'LungCancer': True, 'Smoking': True}, dyspnea)
reject_3 = rejection_sampling('Dyspnea', {'LungCancer': True, 'Smoking': True}, dyspnea, 100000)
weight_3 = likelihood_weighting('Dyspnea', {'LungCancer': True, 'Smoking': True}, dyspnea, 100000)

print("Likelihood of Dyspnea given LungCancer and Smoker using exact inference: ", exact_3[True])
print("Likelihood of Dyspnea given LungCancer and Smoker using approximate rejection sampling: ", reject_3[True])
print("Likelihood of Dyspnea given LungCancer and Smoker using approximate likelihood weighting: ", weight_3[True])


# Expected output - note the sampling methods should return similar but not exact numbers
# Likelihood of Smoker given Tuberculosis using exact inference:  0.5
# Likelihood of Smoker given Tuberculosis using approximate rejection sampling:  0.4874274661508704
# Likelihood of Smoker given Tuberculosis using approximate likelihood weighting:  0.49887711620407665
# Likelihood of Dispnea given VisitToAsia and PositiveXRay using exact inference:  0.6811011940658546
# Likelihood of Dispnea given VisitToAsia and PositiveXRay using approximate rejection sampling:  0.727810650887574
# Likelihood of Dispnea given VisitToAsia and PositiveXRay using approximate likelihood weighting:  0.6789105866434472
# Likelihood of Dispnea given LungCancer and Smoker using exact inference:  0.8200000000000001
# Likelihood of Dispnea given LungCancer and Smoker using approximate rejection sampling:  0.8162444712505026
# Likelihood of Dispnea given LungCancer and Smoker using approximate likelihood weighting:  0.8185400000002676


Likelihood of Smoker given Tuberculosis using exact inference:  0.5
Likelihood of Smoker given Tuberculosis using approximate rejection sampling:  0.5070993914807302
Likelihood of Smoker given Tuberculosis using approximate likelihood weighting:  0.4979386607074076
Likelihood of Dyspnea given VisitToAsia and Positive XRay using exact inference:  0.7143770353395248
Likelihood of Dyspnea given VisitToAsia and Positive XRay using approximate rejection sampling:  0.6643835616438356
Likelihood of Dyspnea given VisitToAsia and Positive XRay using approximate likelihood weighting:  0.7098027061453499
Likelihood of Dyspnea given LungCancer and Smoker using exact inference:  0.9
Likelihood of Dyspnea given LungCancer and Smoker using approximate rejection sampling:  0.9016590045972417
Likelihood of Dyspnea given LungCancer and Smoker using approximate likelihood weighting:  0.9003500000001374
